# Multi-tone Gates Robust to Optical Crosstalk

We consider gates where we have $D$ pairs of tones $\omega_0 \pm \omega_s$ for $s \in \{1...D\}$ and can set the amplitudes $r_s$ of these pairs individually. All these lasers are directed at only a pair of target ions, but some level of optical crosstalk may be incident on the neighboring ions. We want to set the amplitudes $r_s$ such that an $XX$ operation is produced between the target ions but no (or little) $XX$ rotation occurs between any pair of target and nearest neighbor ions.

## Code setup

In [1]:
import numpy as np
from numpy import sin, cos, exp, sqrt, angle
import scipy.integrate as integrate
import scipy.linalg as linalg
import matplotlib.pyplot as plt

import mystic.symbolic as ms
import mystic.solvers as my
import mystic.math as mm

In [2]:
import importlib
import sys, os
from pathlib import Path
sys.path.append('tools/')
import IonChainTools as ict
import MSFidelityEstimation as msfe
importlib.reload(msfe)
from MSFidelityEstimation import *

In [3]:
π = np.pi

## Functions

In [4]:
def calcModeClosureBasis(modes, tonefreqs, T):
    N = len(modes)
    L = []
    for mode in modes:
        νm = mode.freq
        linear_req = np.array([(exp(1j*(ωs-νm)*T)-1)/(ωs-νm) for ωs in tonefreqs])
        L.append(linear_req.real)
        L.append(linear_req.imag)
    L = np.array(L)
    K = linalg.null_space(L).T
    return K

In [5]:
def calcRequiredModeContributions(N, targets):
    modes = ict.calcRadialModes(N,νratio=10)
    neighbors = set([i for i in range(N) if i in (targets[0]-1,targets[0]+1,targets[1]-1,targets[1]+1) and i not in targets])
    # # Remove symmetric neighbors
    # symmetric_neighbors = []
    # for n in neighbors.copy():
    #     for ti in (0,1):
    #         if abs(targets[ti]-n)==1:
    #             if N-1-n == targets[(ti+1)%2]:
    #                 symmetric_neighbors.append(n)
    # print(neighbors)
    # neighbors -= set(symmetric_neighbors)
    # print(neighbors)
    xtalkpairs = list(itertools.product(targets, neighbors))
    A = np.zeros((N,len(xtalkpairs)))
    for m in range(N):
        for p, pair in enumerate(xtalkpairs):
            mvec = modes[m][1]
            A[m,p] = mvec[pair[0]]*mvec[pair[1]]
    nullspace = linalg.null_space(A.T).T
    targetpair_LD_vec = [modes[m][1][targets[0]]*modes[m][1][targets[1]] for m in range(N)]
    targetneighbor_LD_vec = [modes[m][1][xtalkpairs[0][0]]*modes[m][1][xtalkpairs[0][1]] for m in range(N)]
    valid_nullvecs = []
    for nullvec in nullspace:
        #print(nullvec, nullvec@targetpair_LD_vec, nullvec@targetneighbor_LD_vec)
        if abs(nullvec@targetpair_LD_vec) > 0.0001:
            valid_nullvecs.append(nullvec)
    return valid_nullvecs

In [6]:
def generateBmatrix(devicespec, m, t, toneωs):
    N = devicespec.N
    D = len(toneωs)
    Bm = np.zeros((D,D))
    νm = devicespec.modes('radial')[m].freq
    for s1 in range(D):
        ωs1 = toneωs[s1]
        for s2 in range(D):
            ωs2 = toneωs[s2]
            if s1 == s2:
                Bm[s1,s1] = -1/4*1/(νm-ωs2)*(sin((ωs1-νm)*t)/(ωs1-νm) - t)
            else:
                Bm[s1, s2] = -1/4*1/(νm-ωs2)*(sin((ωs1-νm)*t)/(ωs1-νm) - sin((ωs1-ωs2)*t)/(ωs1-ωs2))
    return Bm

In [7]:
def generateQuadraticConstraint(A, ϕ, varname):
    constraint = ""
    for i in range(len(A)):
        for j in range(len(A)):
            constraint+=f"+{A[i,j]}*{varname}{i}*{varname}{j} "
    constraint += f"== {ϕ}"
    return constraint

## Solving for Gate Parameters

In [8]:
from scipy.constants import physical_constants
M=40*physical_constants['atomic mass constant'][0]

qbit_splitting = 2*π*4.1115503520574475e14 # 729nm
#raman_wavelen = 355*1e-9 # 355nm, from "frequency robust ..." paper
#raman_freq = 2*π*scipy.constants.c/raman_wavelen
kvec = qbit_splitting/scipy.constants.c

νz = 2*π*0.32*1e6 # 0.52MHz + 0.5MHz to make radial mode spacing larger
νr = 2*π*2.296*1e6 # 2.296MHz

N = 4

D = 2*N+5
T = 300e-6

targets=(0,1)

trapspec = TrapSpec(νz, νr)
devicespec = TIDeviceSpec(N, trapspec, qbit_splitting, M=M)

In [9]:
# Tones might be aranged as harmonics
tones = [2*π/T*s for s in range(1,D)]

In [10]:
modes = devicespec.modes('radial')
K = calcModeClosureBasis(modes, tones, T)
l = len(K)
χvals = calcRequiredModeContributions(N, targets)[0]
constraints = ""
for m in range(1):
    B = generateBmatrix(devicespec, m, T, tones)
    constraints += generateQuadraticConstraint(K@B@K.T, χvals[m], 'x')+"\n"
cons = ms.generate_constraint(ms.generate_solvers(ms.simplify(constraints)))
objective_func = lambda x, sign=1: sign*abs((K.T@x)@np.ones(len(K.T)))
bounds = [(0., None)]*l
x0 = np.random.rand(l)
sol = my.fmin_powell(objective_func, x0, constraint=cons, disp=True,
                     bounds=bounds, gtol=3, ftol=1e-6, full_output=True,
                     args=(-1,))
sol

Optimization terminated successfully.
         Current function value: -2.1166039800846193e-06
         Iterations: 6
         Function evaluations: 541


(array([7.24065077e-04, 9.99999999e+02, 9.99999979e+02, 9.99999684e+02]),
 -2.1166039800846193e-06,
 6,
 541,
 0,
 array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]))

In [11]:
tone_strengths = K.T@sol[0]
for m in range(N):
    B = generateBmatrix(devicespec, m, T, tones)
    print(tone_strengths.T@B@tone_strengths, χvals[m])

1.6419599573106783e-05 -0.1106457184418904
1.6121590897899777e-05 -0.3100698362936769
1.589360092825641e-05 0.9038523425614734
1.5737078616444913e-05 0.27324927165031443


The tone amplitude optimization currently does not work.